In [1]:
import sys
import re
import numpy as np
import pandas as pd

## Load Data

In [2]:
# Input/output data files
input_file_cases = "processed_individual_cases_Sep20th2020.csv"
# input_file_locations = "processed_location_Sep20th2020.csv"
output_file_cases = "processed_individual_cases_Sep20th2020_1.2_cleaned-and-imputed-age.csv"

In [3]:
# Load data from file(s)
individual_cases = pd.read_csv(input_file_cases)
# locations = pd.read_csv(input_file_locations)

## Data Cleaning

In [4]:
# All NaN
individual_cases.age[individual_cases['age'].isna()]

not_NaN_ages = individual_cases[['age']].dropna()

### Convert range to standard format

In [5]:
"""
Formats like "20-29" are being converted to the standard format.
The strategy here is to take the average of the end points.
"""
range_re = re.compile(r"^(\d+)\s*-\s*(\d+)$")
def range_to_num(s):
    if type(s) == str:
        m = range_re.match(s)
        if m:
            num1, num2 = list(map(float, [x.strip() for x in m.group().split('-')]))
            return (num1 + num2) / 2
        else:
            return s
    else:
        return s

not_NaN_ages['age'] = not_NaN_ages['age'].apply(range_to_num)

### Remove suffix

In [6]:
"""
Remove the '+' or '-' in ages (i.e., convert "65+" to 65.0).
The strategy here is removing the sign directly.
"""
sign_re = re.compile(r"^(\d+(\.\d+)?\+?\-?$)")
def remove_sign(s):
    if type(s) == str:
        m = sign_re.match(s)
        return float(m.group().replace('+', '').replace('-', '')) if m else s
    else:
        return s

not_NaN_ages['age'] = not_NaN_ages['age'].apply(remove_sign)

### Convert month to year

In [7]:
"""
To make sure the unit of age is in year.
Formats like "18 months" are being convert to the standard form (number of years).
"""
month_re = re.compile(r"^(\d+)\s*month")
def month_to_age(s):
    if type(s) == str:
        m = month_re.match(s)
        return float(m.group().split(' ')[0].strip()) / 12.0 if m else s
    else:
        return s

not_NaN_ages['age'] = not_NaN_ages['age'].apply(month_to_age)

### Other formats

In [8]:
"""
If there are any unrecognized formats (i.e., invalid inputs), 
assign NaN to them to ensure the robustness of the program.
"""
# Filter out the inconsistent format
isfloat_re = re.compile(r"^(\d+(\.\d+))$")
def isFloat(s):
    if type(s) == float or type(s) == int:
        return True
    if type(s) == str:
        return True if isfloat_re.match(s) else False
    else:
        return False

# Display if there is any
not_NaN_ages[~not_NaN_ages['age'].apply(isFloat)]

,age


In [9]:
# Assign NaN to them
not_NaN_ages['age'] = not_NaN_ages['age'].apply(lambda x: x if isFloat(x) else np.NaN)

### Convert to float

In [13]:
pd.to_numeric(not_NaN_ages['age']);

In [14]:
# Add the cleaned data back to the original dataset
# Modify the original dataset accordingly
individual_cases['age'] = not_NaN_ages
individual_cases['age']

0          NaN
1         21.0
2         94.0
3          NaN
4          2.0
          ... 
557359    56.0
557360     NaN
557361     NaN
557362    25.0
557363     NaN
Name: age, Length: 557364, dtype: float64

## Impute Missing Values